In [1]:
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import os.path
import time, itertools, re
from sklearn.metrics import make_scorer, balanced_accuracy_score
import miceforest as mf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,StratifiedGroupKFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.mixture import GaussianMixture
from FigPlotting import clusterPlot, plot_dendrogram
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)

/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/


In [2]:
colsLAV=['Lav-Neg_Waardering', 'Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid']
colsSQ48=['SQ48-Vijandigheid','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie', 'SQ48-Cognitieve_Klachten',
          'SQ48-Somatische_Klachten', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme', 'SQ48-Werk_Studie']
colsMHCSF=['MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB']
cols2consider=['Main-Age','Main-ED_Codes', 'EDEQ-Score']+colsLAV+colsSQ48+colsMHCSF
cols2train=['Main-Age']+colsLAV+colsSQ48+colsMHCSF
colNames=['Main-Age', 'Main-ED_Codes', 'EDEQ-Score','Lav-Neg_Waardering','Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid', 
'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 
'MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB'] 
colRename_dict=dict(zip(cols2consider, colNames))
df_adapted_combo=pd.read_csv(fileloc_data+'maskedDAIsy_MainED_Lav_SQ48_MHCSF_Visit1.tsv', sep='\t', decimal=',')
df_adapted_combo['EDtype'] = df_adapted_combo['EDtype'].astype('category')
df_adapted_combo.astype(dict(zip(colsLAV, ['float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsSQ48,['float', 'float','float','float', 'float','float','float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsMHCSF, ['float', 'float','float']))).dtypes
df_adapted_combo.rename(columns=colRename_dict, inplace=True)
adapted_combo_cols=['Main-Age','Lav-Alg_Ontevredenheid', 'Lav-Gebrek_Vertrouwdheid','Lav-Neg_Waardering',
    'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
    'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 'MHCSF-EmotionWB', 
    'MHCSF-SocialWB', 'MHCSF-PsychWB']
#df_adapted_combo['ED_Codes'] = df_adapted_combo['EDtype'].cat.codes

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis,)
from sklearn.svm import SVC

df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Train'].to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Test'].to_numpy()


In [5]:
rf = RandomForestClassifier(criterion="gini")
pipe_RF_search = Pipeline(steps=[('scaler',  StandardScaler()),
            ('RandomForestClassifier', RandomForestClassifier(criterion="gini"))], verbose=0)
rf_param_grid = { "RandomForestClassifier__n_estimators"      : [100, 250, 300, 500],           
           "RandomForestClassifier__max_features"      : [5,7,11,15]}
rf_grid_search = GridSearchCV(pipe_RF_search, rf_param_grid,cv=3, scoring='balanced_accuracy')
rf_grid_search.fit(df_train_adapted.to_numpy(), YTrain)
df_rf_gs = pd.DataFrame(rf_grid_search.cv_results_)[
    ["param_RandomForestClassifier__n_estimators", "param_RandomForestClassifier__max_features","mean_test_score", ]]

df_rf_gs = df_rf_gs.rename(columns={"param_RandomForestClassifier__n_estimators": "n_Trees",
"param_RandomForestClassifier__max_features": "Max_Features", "mean_test_score": "Bal. Acc"})
df_rf_gs.sort_values(by=["Bal. Acc"], ascending=[False]).head(5)

,n_Trees,Max_Features,Bal. Acc
7,500,7,0.366851
9,250,11,0.361391
5,250,7,0.356947
3,500,5,0.352280
4,100,7,0.352245


In [ ]:
pipe_KNN_search = Pipeline(steps=[('scaler',  StandardScaler()),
            ('KNeighborsClassifier', KNeighborsClassifier())], verbose=0)
rf_param_grid = { "KNeighborsClassifier__n_estimators"      : [100, 250, 300, 500],           
           "KNeighborsClassifier__max_features"      : [5,7,11,15]}
rf_grid_search = GridSearchCV(pipe_RF_search, rf_param_grid,cv=3, scoring='balanced_accuracy')
rf_grid_search.fit(df_train_adapted.to_numpy(), YTrain)
df_rf_gs = pd.DataFrame(rf_grid_search.cv_results_)[
    ["param_RandomForestClassifier__n_estimators", "param_RandomForestClassifier__max_features","mean_test_score", ]]

df_rf_gs = df_rf_gs.rename(columns={"param_RandomForestClassifier__n_estimators": "n_Trees",
"param_RandomForestClassifier__max_features": "Max_Features", "mean_test_score": "Bal. Acc"})
df_rf_gs.sort_values(by=["Bal. Acc"], ascending=[False]).head(5)

In [5]:


names = ["KNN", "Linear SVM", "RBF SVM", "RF", "NB","QDA", "LDA"]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]




Index(['Lav-Alg_Ontevredenheid', 'Lav-Gebrek_Vertrouwdheid',
       'Lav-Neg_Waardering', 'Lav-Score', 'intid', 'p_lav_filled_date',
       'p_lav_send_date', 'seq_num-lav', 'MHCSF-EmotionWB', 'MHCSF-PsychWB',
       'MHCSF-Score', 'MHCSF-SocialWB', 'p_mhcsf_filled_date', 'seq_num-mhc',
       'pid', 'seq_num-sq48', 'Split', 'p_sq48_filled_date',
       'SQ48-Vijandigheid', 'SQ48-Agorafobie', 'SQ48-Angst', 'SQ48-Depressie',
       'SQ48-Cognitieve_Klachten', 'SQ48-Somatische_Klachten',
       'SQ48-Sociale_Fobie', 'SQ48-Vitaliteit_Optimisme', 'SQ48-Werk_Studie',
       'SQ48-Score', 'EDEQ-Score', 'EDtype', 'Main-Age', 'Main-Bsex',
       'Main-Highest_Edu', 'edu_level', 'p_dischargedate', 'p_startdate',
       'seq_num-edeq', 'Main-Biosex', 'ED_Codes', 'Main-Education'],
      dtype='object')


NameError: name 'colRename_dict' is not defined